In [162]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
import re

In [ ]:
ruta = Path("D:\\IGAC\\2INivelacion\\Original\\2007\\2007_L2_Buenaventura_Cali\\InfOriginal\\Crudos\\05112007\\JORGE")
ruta2 = Path("G:\\Otros ordenadores\\Mi PC\\IGAC\\archivosPrueba\\Archivos_Pruebas\\L5_GSI_1\\CRUDOS_L5")
ruta3 = Path("D:\\IGAC\\2INivelacion\\Estructurada\\Linea_2\\InfoCalculos\\Crudos\\2008")

# Diccionarios

In [134]:
# Es el diccionario que permitirá saber el método de medición
dict_medicion = {
    "1" : "EF",
    "2" : "EFFE"
}

# Factores de escala
dict_escala = {
    "0" : 1e3,
    "6" : 1e4,
    "8" : 1e5
}

dict_patrones = {
    "ini" : r"^\*?11\d{4}[+-](\S{8,16})\s(83\.\.\d{2})([+-]\d{8,16})", 
    "fin" : r"^\*?11\d{4}[+-](\S{8,16})\s+(573\.+\d{1,2})([+-]\d{8,16})\s(574\.+\d{1,2})([+-]\d{8,16})\s(83\.+\d{1,2})([+-]\d{8,16})",
    "med" : r"^\*?41\d{4}[+-](?:\d{8,16})?\?\.+\d$"
}

# Almacenamiento Observaciones

In [ ]:
@dataclass
class Obs_Niv:
    nom_ini : str
    nom_fin : str
    dif_dist : float
    dist_tot : float
    dif_alt : float
    alt_ini : float
    tipo_med : str
    archivo : str

class Parser_Datos_Niv:
    def __init__(self, patterns, escalas, mediciones):
        self.patterns = patterns
        self.escalas = escalas
        self.mediciones = mediciones
        self._reset()

    def _reset(self):
        self.ini = None
        self.fin = None
        self.dif_dist = None
        self.dist_tot = None
        self.dif_alt = None
        self.alt_ini = None
        self.tipo_med = None
        self._escala = None
        self.archivo = None

    def builder(self):
        if all(v is not None for v in (
            self.ini, self.fin, self.dif_dist,
            self.dist_tot, self.dif_alt,
            self.alt_ini, self.tipo_med
        )):
            assert self.ini is not None
            assert self.fin is not None
            assert self.dif_dist is not None
            assert self.dist_tot is not None
            assert self.dif_alt is not None
            assert self.alt_ini is not None
            assert self.tipo_med is not None
            assert self.archivo is not None

            obs = Obs_Niv(
                nom_ini=self.ini,
                nom_fin=self.fin,
                dif_dist=self.dif_dist,
                dist_tot=self.dist_tot,
                dif_alt=self.dif_alt,
                alt_ini=self.alt_ini,
                tipo_med=self.tipo_med,
                archivo=self.archivo
            )
            self._reset()
            return obs
        return None

    def datos_gsi(self, lin_ant, lin_act, file):
        if lin_act is None:
            return None

        self.archivo = file.name

        search_ini = re.search(self.patterns["ini"], lin_act)
        search_med = re.search(self.patterns["med"], lin_act)

        if search_ini:
            self.ini = search_ini.group(1).lstrip("0")
            self._escala = self.escalas.get(search_ini.group(2)[-1])
            if self._escala:
                self.alt_ini = float(search_ini.group(3)) / self._escala

        if search_med:
            search_fin = re.search(self.patterns["fin"], lin_ant or "")
            if not search_fin or not self._escala:
                return None

            self.fin = search_fin.group(1).lstrip("0")
            self.tipo_med = self.mediciones.get(lin_act[-1])

            self.dif_dist = float(search_fin.group(3)) / self._escala
            self.dist_tot = float(search_fin.group(5)) / self._escala
            self.dif_alt = float(search_fin.group(7)) / self._escala

            return self.builder()

        return None

In [160]:
class Proc_Carp_Niv:
    def __init__(self, parser):
        self.parser = parser
    
    def procesar_carpeta(self, carpeta):
        observaciones = []
        for archivo in carpeta.iterdir():
            self.parser._reset()
            if not archivo.is_file():
                continue

            with archivo.open("r") as f:
                lineas = f.readlines()

            for i, linea in enumerate(lineas):
                lin_ant = lineas[i - 1].strip() if i > 0 else None
                lin_act = linea.strip()
                obs = self.parser.datos_gsi(lin_ant, lin_act, archivo)

                if obs:
                    observaciones.append(obs)

        return observaciones

In [ ]:
parser = Parser_Datos_Niv(
    patterns   = dict_patrones,
    escalas    = dict_escala,
    mediciones = dict_medicion
)

procesador = Proc_Carp_Niv(parser)
observaciones = procesador.procesar_carpeta(ruta3)
df = pd.DataFrame([obs.__dict__ for obs in observaciones])
df["alt_final"] = df["dif_alt"]-df["alt_ini"]
df = df[["nom_ini","nom_fin","alt_final","dist_tot","tipo_med","archivo"]].sort_values(by="archivo").reset_index(drop=True)
df.to_excel("../LeicaGSI_LS10_Parser/data/prueba_crudos.xlsx",index=False)